# Google Flood Forecast API

Exploring basic functionality of the Flood Forecast API. See the following resources: 

- https://colab.research.google.com/drive/1et0vjN9coLck11YhORyprgtOvCuaErVd?usp=sharing#scrollTo=omVd-Y7OJePs
- https://developers.google.com/flood-forecasting/rest/v1/gauges/queryGaugeForecasts
- https://developers.google.com/flood-forecasting

In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [46]:
from dotenv import load_dotenv
import datetime
import os
import requests
import pandas as pd
from src.datasources import grrr

Lets start by getting some metadata for our gauge. See [here](https://developers.google.com/flood-forecasting/rest/v1/floodStatus#resource:-floodstatus) for response schema details.

In [47]:
flood_status = requests.get(
    "https://floodforecasting.googleapis.com/v1/floodStatus:queryLatestFloodStatusByGaugeIds",
    params={
        "key": grrr.FLOODS_API_KEY,
        "gaugeIds": grrr.HYBAS_ID,
    },
).json()

We can see some basic information here about the flood status at this gauge.
As of the time this update was issued, there is no flooding at this gauge. 

In [44]:
flood_status["floodStatuses"]

[{'gaugeId': 'hybas_1120842550',
  'issuedTime': '2025-05-13T19:49:28.775471Z',
  'forecastTimeRange': {'start': '2025-05-16T00:00:00Z',
   'end': '2025-05-17T00:00:00Z'},
  'forecastTrend': 'NO_CHANGE',
  'severity': 'NO_FLOODING',
  'source': 'HYBAS',
  'gaugeLocation': {'latitude': 9.393749999998931,
   'longitude': 12.368749999996965},
  'qualityVerified': True}]

Now imagine we want to check the latest forecasts for our gauge. 

In [67]:
today = datetime.datetime.now()

res = requests.get(
    "https://floodforecasting.googleapis.com/v1/gauges:queryGaugeForecasts",
    params={
        "key": grrr.FLOODS_API_KEY,
        "gaugeIds": grrr.HYBAS_ID,
        "issuedTimeStart": today.strftime("%Y-%m-%d"),
    },
).json()

In [68]:
rows = []

for forecast in res["forecasts"][HYBAS_ID]["forecasts"]:
    issued_time = forecast["issuedTime"]
    gauge_id = forecast["gaugeId"]

    # Process each forecast range
    for range_item in forecast["forecastRanges"]:
        row = {
            "issuedTime": issued_time,
            "gaugeId": gauge_id,
            "value": range_item["value"],
            "forecastStartTime": range_item["forecastStartTime"],
            "forecastEndTime": range_item["forecastEndTime"],
        }
        rows.append(row)

# Create the DataFrame
df = pd.DataFrame(rows)

# Convert string timestamps to datetime objects for easier analysis
df["issuedTime"] = pd.to_datetime(df["issuedTime"])
df["forecastStartTime"] = pd.to_datetime(df["forecastStartTime"])
df["forecastEndTime"] = pd.to_datetime(df["forecastEndTime"])

Note that the validity time for some of the forecast data is *before* the issue time? See note from the docs [here](https://developers.google.com/flood-forecasting/rest/v1/gauges/queryGaugeForecasts#forecast). 

> Note: Some of the forecast ranges can potentially be earlier than the issued time. This can happen due to e.g., lags in input data for the model. With the above example, it could be that the issue time is 5pm, and the forecast ranges are for 4pm, 5pm, 6pm, etc.

In [69]:
df

,issuedTime,gaugeId,value,forecastStartTime,forecastEndTime
0,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,59.300571,2025-05-11 00:00:00+00:00,2025-05-12 00:00:00+00:00
1,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,59.048435,2025-05-12 00:00:00+00:00,2025-05-13 00:00:00+00:00
2,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,57.622623,2025-05-13 00:00:00+00:00,2025-05-14 00:00:00+00:00
3,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,57.238838,2025-05-14 00:00:00+00:00,2025-05-15 00:00:00+00:00
4,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,57.178448,2025-05-15 00:00:00+00:00,2025-05-16 00:00:00+00:00
5,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,57.622551,2025-05-16 00:00:00+00:00,2025-05-17 00:00:00+00:00
6,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,57.979713,2025-05-17 00:00:00+00:00,2025-05-18 00:00:00+00:00
7,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,57.433430,2025-05-18 00:00:00+00:00,2025-05-19 00:00:00+00:00
8,2025-05-13 19:49:28.775471+00:00,hybas_1120842550,59.632648,2025-05-12 00:00:00+00:00,2025-05-13 00:00:00+00:00
9,2025-05-13 19:49:28.775471+00:00,hybas_1120842550,59.000004,2025-05-13 00:00:00+00:00,2025-05-14 00:00:00+00:00


Let's add a lead time as well

In [65]:
# Calculate leadtime in hours (time between when forecast was issued and the forecast start time)
df["leadtime_hours"] = (
    df["forecastStartTime"] - df["issuedTime"]
).dt.total_seconds() / 3600
df["leadtime_days"] = (
    df["forecastStartTime"] - df["issuedTime"]
).dt.total_seconds() / (24 * 3600)
df = df.sort_values(["issuedTime", "forecastStartTime"])

The timing of when forecasts are issued is also interesting. It looks like there were two separate forecasts issued today. Would this be the same issue as above relating to lags in input data for the model?

In [66]:
df

,issuedTime,gaugeId,value,forecastStartTime,forecastEndTime,leadtime_hours,leadtime_days
0,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,59.300571,2025-05-11 00:00:00+00:00,2025-05-12 00:00:00+00:00,-61.712095,-2.571337
1,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,59.048435,2025-05-12 00:00:00+00:00,2025-05-13 00:00:00+00:00,-37.712095,-1.571337
2,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,57.622623,2025-05-13 00:00:00+00:00,2025-05-14 00:00:00+00:00,-13.712095,-0.571337
3,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,57.238838,2025-05-14 00:00:00+00:00,2025-05-15 00:00:00+00:00,10.287905,0.428663
4,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,57.178448,2025-05-15 00:00:00+00:00,2025-05-16 00:00:00+00:00,34.287905,1.428663
5,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,57.622551,2025-05-16 00:00:00+00:00,2025-05-17 00:00:00+00:00,58.287905,2.428663
6,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,57.979713,2025-05-17 00:00:00+00:00,2025-05-18 00:00:00+00:00,82.287905,3.428663
7,2025-05-13 13:42:43.540544+00:00,hybas_1120842550,57.433430,2025-05-18 00:00:00+00:00,2025-05-19 00:00:00+00:00,106.287905,4.428663
8,2025-05-13 19:49:28.775471+00:00,hybas_1120842550,59.632648,2025-05-12 00:00:00+00:00,2025-05-13 00:00:00+00:00,-43.824660,-1.826027
9,2025-05-13 19:49:28.775471+00:00,hybas_1120842550,59.000004,2025-05-13 00:00:00+00:00,2025-05-14 00:00:00+00:00,-19.824660,-0.826027
